In [2]:
import os
import getpass

import pinecone
from langchain.retrievers import PineconeHybridSearchRetriever

/mnt/c/Users/marcelom/repos/auto-evaluator/api/venv-auto-evaluator/lib/python3.10/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
os.environ["PINECONE_API_KEY"] = getpass.getpass("Pinecone API Key:")

In [4]:
os.environ["PINECONE_ENVIRONMENT"] = getpass.getpass("Pinecone Environment:")

In [5]:
api_key = os.getenv("PINECONE_API_KEY") or "PINECONE_API_KEY"
env = os.getenv("PINECONE_ENVIRONMENT") or "PINECONE_ENVIRONMENT"

index_name = "auto-evaluator-test"

pinecone.init(api_key=api_key, environment=env)
pinecone.whoami()

WhoAmIResponse(username='0c1131f', user_label='default', projectname='e7026e9')

In [7]:
# create the index
pinecone.create_index(
    name=index_name,
    dimension=1536,  # dimensionality of dense model
    metric="dotproduct",  # sparse values supported only for dotproduct
    pod_type="s1",
    metadata_config={"indexed": []},  # see explaination above
)

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=UTF-8', 'date': 'Wed, 12 Jul 2023 17:46:59 GMT', 'x-envoy-upstream-service-time': '630', 'content-length': '131', 'server': 'envoy'})
HTTP response body: The index exceeds the project quota of 6 pods by 1 pods. Upgrade your account or change the project settings to increase the quota.


### Embeddings OpenAI

In [19]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
openai.api_type = "azure"
openai.api_key = os.environ.get("AZURE_OPENAI_API_KEY")
openai.api_base = os.environ.get("AZURE_OPENAI_API_BASE")
openai.api_version = "2023-05-15"

In [26]:
splits = ["Your text string goes here", "123444fergg fergergwgeArithmeticError"]
embeddings = [
    openai.Embedding.create(
        input=split,
        engine="text-embedding-ada-002"
    )['data'][0]['embedding']
    for split in splits
]
print(embeddings)

[[-0.00621345778927207, -0.009322499856352806, 0.03355174884200096, -0.02533499337732792, -0.01605413109064102, -0.0189225934445858, 0.03382933884859085, 0.02837000973522663, 0.01525836531072855, 0.009900818578898907, -0.018043547868728638, 0.015896828845143318, 0.02366943471133709, -0.009058786556124687, 0.026130760088562965, 0.0050383140332996845, -0.016498280689120293, 0.00289622088894248, -0.020190268754959106, 0.024761300534009933, -0.01065031997859478, 0.019579563289880753, -0.017136745154857635, 0.02601972222328186, 0.01915392093360424, 0.006046901922672987, -0.0030789696611464024, 0.026685945689678192, 0.012186335399746895, -0.0015117254806682467, -0.00537142576649785, -0.020153256133198738, 0.040306512266397476, -0.022355495020747185, -0.025149930268526077, 0.021818814799189568, 0.011621896177530289, 0.021948358044028282, 0.008003933355212212, -0.014147993177175522, 0.027629762887954712, -0.002674146555364132, -0.009447417221963406, -0.015804298222064972, -0.02862909808754921,

### Retrieve Information

In [5]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone

import pinecone

In [14]:
os.environ["PINECONE_API_KEY"] = os.environ.get("PINECONE_API_KEY")
os.environ["PINECONE_ENVIRONMENT"] = os.environ.get("PINECONE_ENVIRONMENT")

In [15]:
openai_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
openai_api_base = os.environ.get("AZURE_OPENAI_API_BASE")

embd = OpenAIEmbeddings(
    deployment="text-embedding-ada-002",
    openai_api_key=openai_api_key,
    openai_api_type="azure",
    openai_api_base=openai_api_base,
    openai_api_version="2023-05-15",
    chunk_size=1,
)

In [23]:
api_key=os.environ["PINECONE_API_KEY"]
env=os.environ["PINECONE_ENVIRONMENT"]
pinecone.init(api_key=api_key, environment=env)

In [24]:
index_name = os.environ.get("PINECONE_INDEX")
index = pinecone.Index(index_name)

vectorstore = Pinecone(index, embd.embed_query, text_key='text')
retriever = vectorstore.as_retriever()

In [25]:
query = "o que é o echo dot?"
vectorstore.similarity_search(
    query,
    k=3
)

Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.


[Document(page_content='Após o pareamento com lâmpadas, plugues e outros aparelhos compatíveis, o Echo Dot\npermite o controle da sua casa inteligente por voz, movimentos ou a partir da temperatura\ninterna. Crie rotinas para ligar a luz, o ar condicionado ou reproduzir músicas em horários\nespecíﬁcos quando você entrar em um ambiente ou quando a temperatura do local estiver\ndiferente do ideal.\nPause músicas ou adie alarmes com\num toque\nCom um simples toque, você pode pausar músicas, ﬁnalizar chamadas ou parar timers.\nPrecisa de mais uns minutinhos de sono? Toque para adiar o alarme.06/07/2023, 14:18 Novo Echo Dot 5ª geração | O Echo Dot com o melhor som já lançado | Cor Branca : Amazon.com.br\nhttps://www .amazon.com.br/Echo-Dot-4ª-geração-Cor-Branca/dp/B09B8XVSDP/ref=asc_df_B09B8XVSDP/?tag=googleshopp00-20&linkCode= … 7/13', metadata={'id': '4a4239b8-34c7-44e8-9afb-b52c70c11d69', 'source_id': 'auto_evaluator_test123'})]

In [26]:
query = "o que é o echo dot?"
retriever.get_relevant_documents(query)

Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.


[Document(page_content='Após o pareamento com lâmpadas, plugues e outros aparelhos compatíveis, o Echo Dot\npermite o controle da sua casa inteligente por voz, movimentos ou a partir da temperatura\ninterna. Crie rotinas para ligar a luz, o ar condicionado ou reproduzir músicas em horários\nespecíﬁcos quando você entrar em um ambiente ou quando a temperatura do local estiver\ndiferente do ideal.\nPause músicas ou adie alarmes com\num toque\nCom um simples toque, você pode pausar músicas, ﬁnalizar chamadas ou parar timers.\nPrecisa de mais uns minutinhos de sono? Toque para adiar o alarme.06/07/2023, 14:18 Novo Echo Dot 5ª geração | O Echo Dot com o melhor som já lançado | Cor Branca : Amazon.com.br\nhttps://www .amazon.com.br/Echo-Dot-4ª-geração-Cor-Branca/dp/B09B8XVSDP/ref=asc_df_B09B8XVSDP/?tag=googleshopp00-20&linkCode= … 7/13', metadata={'id': '4a4239b8-34c7-44e8-9afb-b52c70c11d69', 'source_id': 'auto_evaluator_test123'})]